In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


Mounted at /content/drive


In [ ]:
data_path = '/content/drive/MyDrive/sales_data_with_issues.csv'
data = pd.read_csv(data_path)
data

,Category_Technology,Category_Office Supplies,Category_Furniture,City,Sales
0,0,0,1,washington,832.698188
1,1,0,0,houston,832.584084
2,0,0,1,los angeles,793.030324
3,0,0,1,phoenix,876.387212
4,0,1,0,new york,847.019964
...,...,...,...,...,...
5005,0,1,0,phoenix,926.859340
5006,1,0,0,san jose,981.280420
5007,0,1,0,san francisco,752.365337
5008,1,0,0,denver,894.706730


In [ ]:
unique_cities = data['City'].unique()

print("Unique cities in the dataset:")
for city in unique_cities:
    print(city)


Unique cities in the dataset:
washington
houston
los angeles
phoenix
new york
philadelphia
san antonio
san diego
charlotte
fort worth
seattle
san jose
jacksonville
nan
chicago
denver
dallas
columbus
san francisco
austin
indianapolis
CHARLOTTE
PHILADELPHIA
JACKSONVILLE
FORT WORTH
AUSTIN
SAN JOSE
PHOENIX
SEATTLE
DENVER
DALLAS
INDIANAPOLIS
HOUSTON


In [ ]:
data['Sales'].fillna(data['Sales'].median(), inplace=True)  # Replace missing sales with median
data['City'].fillna('Unknown', inplace=True)  # Replace missing cities with 'Unknown'
data.drop_duplicates(inplace=True)  # Remove duplicates
data['City'] = data['City'].str.lower().str.strip()  # Standardize text formatting


# Handle outliers in 'Sales' column
q1 = data['Sales'].quantile(0.25)
q3 = data['Sales'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
data['Sales'] = np.where(data['Sales'] > upper_bound, upper_bound, data['Sales'])
data['Sales'] = np.where(data['Sales'] < lower_bound, lower_bound, data['Sales'])

print("Data cleaning complete.")
data.head()


Data cleaning complete.


<ipython-input-4-74b2d1081f56>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Sales'].fillna(data['Sales'].median(), inplace=True)  # Replace missing sales with median
<ipython-input-4-74b2d1081f56>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'd

,Category_Technology,Category_Office Supplies,Category_Furniture,City,Sales
0,0,0,1,washington,832.698188
1,1,0,0,houston,832.584084
2,0,0,1,los angeles,793.030324
3,0,0,1,phoenix,876.387212
4,0,1,0,new york,847.019964


In [ ]:
# Replace NaN cities with 'Unknown'
data['City'] = data['City'].fillna('Unknown')

# One-hot encode the 'City' column
data = pd.get_dummies(data, columns=['City'], drop_first=True)

print("Feature engineering complete. NaN cities replaced with 'Unknown'.")
data.head()


Feature engineering complete. NaN cities replaced with 'Unknown'.


,Category_Technology,Category_Office Supplies,Category_Furniture,Sales,City_charlotte,City_chicago,City_columbus,City_dallas,City_denver,City_fort worth,...,City_new york,City_philadelphia,City_phoenix,City_san antonio,City_san diego,City_san francisco,City_san jose,City_seattle,City_unknown,City_washington
0,0,0,1,832.698188,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1,0,0,832.584084,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0,0,1,793.030324,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0,0,1,876.387212,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
4,0,1,0,847.019964,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False


In [ ]:
data.columns

Index(['Category_Technology', 'Category_Office Supplies', 'Category_Furniture',
       'Sales', 'City_charlotte', 'City_chicago', 'City_columbus',
       'City_dallas', 'City_denver', 'City_fort worth', 'City_houston',
       'City_indianapolis', 'City_jacksonville', 'City_los angeles',
       'City_new york', 'City_philadelphia', 'City_phoenix',
       'City_san antonio', 'City_san diego', 'City_san francisco',
       'City_san jose', 'City_seattle', 'City_unknown', 'City_washington'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()

In [ ]:
X = data.drop(columns=['Sales'])
y = data['Sales']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Linear Regression
print("Training Linear Regression...")
linear_reg = LinearRegression()
linear_reg.fit(X_train_scaled, y_train)
linear_reg_pred = linear_reg.predict(X_test_scaled)

linear_reg_rmse = np.sqrt(mean_squared_error(y_test, linear_reg_pred))
linear_reg_mae = mean_absolute_error(y_test, linear_reg_pred)
linear_reg_r2 = r2_score(y_test, linear_reg_pred)
print(f"Linear Regression -> RMSE: {linear_reg_rmse}, MAE: {linear_reg_mae}, R²: {linear_reg_r2}")

# Random Forest
print("\nTraining Random Forest...")
random_forest = RandomForestRegressor(random_state=42)
random_forest.fit(X_train_scaled, y_train)
random_forest_pred = random_forest.predict(X_test_scaled)

random_forest_rmse = np.sqrt(mean_squared_error(y_test, random_forest_pred))
random_forest_mae = mean_absolute_error(y_test, random_forest_pred)
random_forest_r2 = r2_score(y_test, random_forest_pred)
print(f"Random Forest -> RMSE: {random_forest_rmse}, MAE: {random_forest_mae}, R²: {random_forest_r2}")

# Gradient Boosting
print("\nTraining Gradient Boosting...")
gradient_boosting = GradientBoostingRegressor(random_state=42)
gradient_boosting.fit(X_train_scaled, y_train)
gradient_boosting_pred = gradient_boosting.predict(X_test_scaled)

gradient_boosting_rmse = np.sqrt(mean_squared_error(y_test, gradient_boosting_pred))
gradient_boosting_mae = mean_absolute_error(y_test, gradient_boosting_pred)
gradient_boosting_r2 = r2_score(y_test, gradient_boosting_pred)
print(f"Gradient Boosting -> RMSE: {gradient_boosting_rmse}, MAE: {gradient_boosting_mae}, R²: {gradient_boosting_r2}")

# XGBoost
print("\nTraining XGBoost...")
xgboost = XGBRegressor(random_state=42)
xgboost.fit(X_train_scaled, y_train)
xgboost_pred = xgboost.predict(X_test_scaled)

xgboost_rmse = np.sqrt(mean_squared_error(y_test, xgboost_pred))
xgboost_mae = mean_absolute_error(y_test, xgboost_pred)
xgboost_r2 = r2_score(y_test, xgboost_pred)
print(f"XGBoost -> RMSE: {xgboost_rmse}, MAE: {xgboost_mae}, R²: {xgboost_r2}")



Training Linear Regression...
Linear Regression -> RMSE: 52.1306049053111, MAE: 42.384021550732285, R²: 0.5456064376578389

Training Random Forest...
Random Forest -> RMSE: 52.434527375667415, MAE: 42.564138915262525, R²: 0.5402927460817719

Training Gradient Boosting...
Gradient Boosting -> RMSE: 53.14132057415051, MAE: 43.02424038346773, R²: 0.5278159355253216

Training XGBoost...
XGBoost -> RMSE: 52.54837419174375, MAE: 42.6826985130215, R²: 0.5382943291424471


In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Predictions from individual models
linear_pred = linear_reg.predict(X_test_scaled)
random_forest_pred = random_forest.predict(X_test_scaled)
gradient_boosting_pred = gradient_boosting.predict(X_test_scaled)
xgboost_pred = xgboost.predict(X_test_scaled)

# Averaging Ensemble
print("Evaluating Averaging Ensemble...")
ensemble_pred_avg = (linear_pred + random_forest_pred + gradient_boosting_pred + xgboost_pred) / 4

# Evaluate averaging ensemble
rmse_avg = np.sqrt(mean_squared_error(y_test, ensemble_pred_avg))
mae_avg = mean_absolute_error(y_test, ensemble_pred_avg)
r2_avg = r2_score(y_test, ensemble_pred_avg)
print(f"Averaging Ensemble -> RMSE: {rmse_avg}, MAE: {mae_avg}, R²: {r2_avg}")

# Stacking Ensemble
print("\nEvaluating Stacking Ensemble...")
stacked_features = np.column_stack((linear_pred, random_forest_pred, gradient_boosting_pred, xgboost_pred))

# Define and train the meta-model
meta_model = LinearRegression()
meta_model.fit(stacked_features, y_test)

# Predict using the stacking ensemble
ensemble_pred_stacked = meta_model.predict(stacked_features)

# Evaluate stacking ensemble
rmse_stacked = np.sqrt(mean_squared_error(y_test, ensemble_pred_stacked))
mae_stacked = mean_absolute_error(y_test, ensemble_pred_stacked)
r2_stacked = r2_score(y_test, ensemble_pred_stacked)
print(f"Stacked Ensemble -> RMSE: {rmse_stacked}, MAE: {mae_stacked}, R²: {r2_stacked}")


Evaluating Averaging Ensemble...
Averaging Ensemble -> RMSE: 52.31268172742016, MAE: 42.50382500198412, R²: 0.5424267692219841

Evaluating Stacking Ensemble...
Stacked Ensemble -> RMSE: 52.07437524701739, MAE: 42.40137637476694, R²: 0.5465861544699011


In [ ]:
import joblib

# Save the individual models
joblib.dump(linear_reg, "linear_reg.pkl")
print("Linear Regression model saved as linear_reg.pkl")

joblib.dump(random_forest, "random_forest.pkl")
print("Random Forest model saved as random_forest.pkl")

joblib.dump(gradient_boosting, "gradient_boosting.pkl")
print("Gradient Boosting model saved as gradient_boosting.pkl")

joblib.dump(xgboost, "xgboost.pkl")
print("XGBoost model saved as xgboost.pkl")

# Save the meta-model for stacking
joblib.dump(meta_model, "meta_model.pkl")
print("Stacking meta-model saved as meta_model.pkl")

# Save the scaler
joblib.dump(scaler, "scaler.pkl")
print("Scaler saved as scaler.pkl")


Linear Regression model saved as linear_reg.pkl
Random Forest model saved as random_forest.pkl
Gradient Boosting model saved as gradient_boosting.pkl
XGBoost model saved as xgboost.pkl
Stacking meta-model saved as meta_model.pkl
Scaler saved as scaler.pkl


In [ ]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import numpy as np
import joblib

# Load models and scaler
try:
    linear_reg = joblib.load("linear_reg.pkl")
    random_forest = joblib.load("random_forest.pkl")
    gradient_boosting = joblib.load("gradient_boosting.pkl")
    xgboost = joblib.load("xgboost.pkl")
    meta_model = joblib.load("meta_model.pkl")
    scaler = joblib.load("scaler.pkl")
    st.sidebar.success("Models loaded successfully.")
except Exception as e:
    st.sidebar.error(f"Error loading models: {e}")
    st.stop()

# Feature input options
cities = [
    "charlotte", "chicago", "columbus", "dallas", "denver", "fort worth",
    "houston", "indianapolis", "jacksonville", "los angeles", "new york",
    "philadelphia", "phoenix", "san antonio", "san diego", "san francisco",
    "san jose", "seattle", "unknown", "washington"
]

categories = [0, 1]  # Options for binary values

# Streamlit app
st.title("Sales Forecasting App")
st.header("Enter the Features for Prediction")

# User input fields
Category_Technology = st.selectbox("Category: Technology (1 for Yes, 0 for No)", categories)
Category_Office_Supplies = st.selectbox("Category: Office Supplies (1 for Yes, 0 for No)", categories)
Category_Furniture = st.selectbox("Category: Furniture (1 for Yes, 0 for No)", categories)
City = st.selectbox("City", cities)

# Predict button
if st.button("Predict Sales"):
    try:
        # Prepare input features
        city_features = [1 if c == City else 0 for c in cities]
        input_features = np.array([[Category_Technology, Category_Office_Supplies, Category_Furniture] + city_features])

        # Debugging: Display input features
        st.write("Input Features:")
        st.write(input_features)
        st.write(f"Shape of Input Features: {input_features.shape}")

        # Scale the features
        scaled_features = scaler.transform(input_features)

        # Generate predictions from base models
        linear_pred = linear_reg.predict(scaled_features)
        random_forest_pred = random_forest.predict(scaled_features)
        gradient_boosting_pred = gradient_boosting.predict(scaled_features)
        xgboost_pred = xgboost.predict(scaled_features)

        # Debugging: Display predictions from base models
        st.write("Base Model Predictions:")
        st.write(f"Linear Regression Prediction: {linear_pred[0]}")
        st.write(f"Random Forest Prediction: {random_forest_pred[0]}")
        st.write(f"Gradient Boosting Prediction: {gradient_boosting_pred[0]}")
        st.write(f"XGBoost Prediction: {xgboost_pred[0]}")

        # Stack predictions for meta-model
        stacked_features = np.column_stack((linear_pred, random_forest_pred, gradient_boosting_pred, xgboost_pred))

        # Debugging: Display stacked features
        st.write("Stacked Features:")
        st.write(stacked_features)
        st.write(f"Shape of Stacked Features: {stacked_features.shape}")

        # Predict using meta-model
        predicted_sales = meta_model.predict(stacked_features)[0]

        # Display the result
        st.success(f"Predicted Sales: ${round(predicted_sales, 2)}")
    except Exception as e:
        st.error(f"Error during prediction: {e}")
""")


In [ ]:
!ngrok authtoken 2pe98ltrGddUEwciPJGU6STpflZ_7wzs5F9LMLgckAWvWwFvE

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!streamlit run app.py &>/dev/null&


In [ ]:
from pyngrok import ngrok

# Expose Streamlit port 8501
public_url = ngrok.connect(addr="8501", proto="http")
print(f"Streamlit app is running at {public_url}")

Streamlit app is running at NgrokTunnel: "https://7136-35-221-128-38.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
from pyngrok import ngrok
ngrok.kill()


In [ ]:
# from pyngrok import ngrok
# public_url = ngrok.connect(addr="8501", proto="http")
# print(f"Streamlit app is running at {public_url}")
